In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import numpy as np
import pandas as pd
import stan
import arviz as az

In [3]:
import mmm_helpers as helpers

In [4]:
# Data    
# Four years' (209 weeks) records of sales, media impression and media spending at weekly level.   
df = pd.read_csv('data.csv')

In [5]:
# 1. media variables
# media impression
mdip_cols=[col for col in df.columns if 'mdip_' in col]
# media spending
mdsp_cols=[col for col in df.columns if 'mdsp_' in col]

In [6]:
# 2. control variables
# macro economics variables
me_cols = [col for col in df.columns if 'me_' in col]
# store count variables
st_cols = ['st_ct']
# markdown/discount variables
mrkdn_cols = [col for col in df.columns if 'mrkdn_' in col]
# holiday variables
hldy_cols = [col for col in df.columns if 'hldy_' in col]
# seasonality variables
seas_cols = [col for col in df.columns if 'seas_' in col]
base_vars = me_cols+st_cols+mrkdn_cols+hldy_cols+seas_cols

In [7]:
# 3. sales variables
sales_cols =['sales']

In [8]:
df_ctrl, sc_ctrl = helpers.mean_center_transform(df, ['sales']+me_cols+st_cols+mrkdn_cols)

In [9]:
df_ctrl = pd.concat([df_ctrl, df[hldy_cols+seas_cols]], axis=1)

In [10]:
# variables positively related to sales: macro economy, store count, markdown, holiday
pos_vars = [col for col in base_vars if col not in seas_cols]
# using tolist ensures we can json serialize
X1 = df_ctrl[pos_vars].values

In [11]:
# variables may have either positive or negtive impact on sales: seasonality
pn_vars = seas_cols
# using tolist ensures we can json serialize
X2 = df_ctrl[pn_vars].values

In [15]:
ctrl_data = {
    # num observations
    'num_obs': len(df_ctrl),
    # num pos control vars
    'num_pos_ctrl': len(pos_vars),
      # num pn control vars
    'num_pn_ctrl': len(pn_vars),
    # pos control values (shape = num_obs,num_ctrl_vars)
    'pos_ctrl_vals': df_ctrl[pos_vars].values,
    # pn control values (shape = num_obs,num_ctrl_vars)
    'pn_ctrl_vals': df_ctrl[pn_vars].values,
    # observed sales
    'y': df_ctrl['sales'].values,
}

In [28]:
ctrl_code = '''
data {
  // the total number of observations
  int num_obs;
  // the vector of sales
  real<lower=0> y[num_obs];
  // num pos ctrl vars
  int num_pos_ctrl;
  // num pn ctrl vars
  int num_pn_ctrl;
  // a matrix of pos control variables
  row_vector[num_pos_ctrl] pos_ctrl_vals[num_obs];
  // a matrix of pos control variables
  row_vector[num_pn_ctrl] pn_ctrl_vals[num_obs];
}

parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // coefficients for pos control variables
  vector<lower=0>[num_pos_ctrl] beta_pos_ctrl;
    // coefficients for pn control variables
  vector[num_pn_ctrl] beta_pn_ctrl;
}

transformed parameters {
  // a vector of the mean response
  real mu[num_obs];
  for (nn in 1:num_obs) {
    mu[nn] <- tau +
      dot_product(pos_ctrl_vals[nn], beta_pos_ctrl) +
      dot_product(pn_ctrl_vals[nn], beta_pn_ctrl);
    }
}

model {
  // PRIORS
  tau ~ normal(0, 5);
  for (ctrl_index in 1:num_pos_ctrl) {
    beta_pos_ctrl[ctrl_index] ~ normal(0,1);
  }
  for (ctrl_index in 1:num_pn_ctrl) {
    beta_pn_ctrl[ctrl_index] ~ normal(0,1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  // The Model
  y ~ normal(mu, sqrt(noise_var));
}
'''

In [30]:
ctrl_posterior = stan.build(ctrl_code,data=ctrl_data,random_seed=1)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_miwck6pv/model_cwntou2c.stan', line 32, column 11: assignment operator <- is deprecated in the Stan language; use = instead.Warning in '/tmp/httpstan_miwck6pv/model_cwntou2c.stan', line 46, column 28: Argument 0.0005 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_miwck6pv/model_cwntou2c.stan', line 46, column 22: Argument 0.05 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).


In [31]:
ctrl_fit = ctrl_posterior.sample(num_chains=4, num_samples=1000)

Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  21% (1700/8000)
Sampling:  22% (1800/8000)
Sampling:  24% (1900/8000)
Sampling:  25% (2000/8000)
Sampling:  26% (2100/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  31% (2500/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  35% (2800/8000)
Sampling:  36% (2900/8000)
Sampling:  38% (3000/8000)
Sampling:  39% (3100/8000)
Sampling:  40% (3200/8000)
Sampling:  41% (3300/8000)
Samplin

In [32]:
ctrl_inference = az.from_pystan(ctrl_fit)

In [33]:
ctrl_inference_summary = az.summary(ctrl_inference)

In [34]:
ctrl_inference_summary

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
noise_var,0.101,0.011,0.082,0.122,0.000,0.000,4781.0,2892.0,1.0
tau,-0.807,0.651,-2.066,0.340,0.017,0.012,1502.0,2320.0,1.0
beta_pos_ctrl[0],0.399,0.318,0.000,0.973,0.006,0.004,2682.0,2220.0,1.0
beta_pos_ctrl[1],0.168,0.129,0.000,0.395,0.002,0.001,2923.0,1949.0,1.0
beta_pos_ctrl[2],0.524,0.293,0.015,1.020,0.006,0.004,2461.0,1760.0,1.0
...,...,...,...,...,...,...,...,...,...
mu[204],1.019,0.128,0.784,1.257,0.002,0.001,4480.0,3519.0,1.0
mu[205],0.939,0.138,0.684,1.195,0.002,0.002,3399.0,3027.0,1.0
mu[206],0.688,0.096,0.518,0.876,0.001,0.001,4275.0,3378.0,1.0
mu[207],0.644,0.096,0.477,0.834,0.001,0.001,4252.0,3702.0,1.0


In [ ]:
az.plot_trace(var_names=

In [16]:
def extract_ctrl_model(fit_result, pos_vars=pos_vars, pn_vars=pn_vars, 
                       extract_param_list=False):
    ctrl_model = {}
    ctrl_model['pos_vars'] = pos_vars
    ctrl_model['pn_vars'] = pn_vars
    ctrl_model['beta1'] = fit_result['beta1'].mean(axis=1).tolist()
    ctrl_model['beta2'] = fit_result['beta2'].mean(axis=1).tolist()
    ctrl_model['alpha'] = fit_result['alpha'].mean()
    if extract_param_list:
        ctrl_model['beta1_list'] = fit_result['beta1'].tolist()
        ctrl_model['beta2_list'] = fit_result['beta2'].tolist()
        ctrl_model['alpha_list'] = fit_result['alpha'].tolist()
    return ctrl_model

In [17]:
base_sales_model = extract_ctrl_model(ctrl_fit, pos_vars=pos_vars, pn_vars=pn_vars)

In [18]:
def ctrl_model_predict(ctrl_model, df):
    pos_vars, pn_vars = ctrl_model['pos_vars'], ctrl_model['pn_vars'] 
    X1, X2 = df[pos_vars].values, df[pn_vars].values
    beta1, beta2 = np.array(ctrl_model['beta1']), np.array(ctrl_model['beta2'])
    alpha = ctrl_model['alpha']
    y_pred = np.dot(X1,beta1) + np.dot(X2,beta2) + alpha
    return y_pred

In [19]:
base_sales = ctrl_model_predict(base_sales_model, df_ctrl)

In [20]:
df['base_sales'] = base_sales*sc_ctrl['sales']
# evaluate control model
print('mape: ',helpers.mean_absolute_percentage_error(df['sales'], df['base_sales']))

mape:  26.165350389170634


In [21]:
# 2.2 Marketing Mix Model
df_mmm, sc_mmm = helpers.mean_log1p_transform(df, ['sales', 'base_sales'])
mu_mdip = df[mdip_cols].apply(np.mean, axis=0).values
max_lag = 8
num_media = len(mdip_cols)
# padding zero * (max_lag-1) rows
X_media = np.concatenate((np.zeros((max_lag-1, num_media)), df[mdip_cols].values), axis=0)
X_ctrl = df_mmm['base_sales'].values.reshape(len(df),1)
mmm_data = {
    'N': len(df),
    'max_lag': max_lag, 
    'num_media': num_media,
    'X_media': X_media, 
    'mu_mdip': mu_mdip,
    'num_ctrl': X_ctrl.shape[1],
    'X_ctrl': X_ctrl, 
    'y': df_mmm['sales'].values
}

In [22]:
mmm_code = '''
functions {
  // the adstock transformation with a vector of weights
  real Adstock(vector t, row_vector weights) {
    return dot_product(t, weights) / sum(weights);
  }
}
data {
  // the total number of observations
  int<lower=1> N;
  // the vector of sales
  real y[N];
  // the maximum duration of lag effect, in weeks
  int<lower=1> max_lag;
  // the number of media channels
  int<lower=1> num_media;
  // matrix of media variables
  matrix[N+max_lag-1, num_media] X_media;
  // vector of media variables' mean
  real mu_mdip[num_media];
  // the number of other control variables
  int<lower=1> num_ctrl;
  // a matrix of control variables
  matrix[N, num_ctrl] X_ctrl;
}
parameters {
  // residual variance
  real<lower=0> noise_var;
  // the intercept
  real tau;
  // the coefficients for media variables and base sales
  vector<lower=0>[num_media+num_ctrl] beta;
  // the decay and peak parameter for the adstock transformation of
  // each media
  vector<lower=0,upper=1>[num_media] decay;
  vector<lower=0,upper=ceil(max_lag/2.0)>[num_media] peak;
}
transformed parameters {
  // the cumulative media effect after adstock
  real cum_effect;
  // matrix of media variables after adstock
  matrix[N, num_media] X_media_adstocked;
  // matrix of all predictors
  matrix[N, num_media+num_ctrl] X;
  
  // adstock, mean-center, log1p transformation
  row_vector[max_lag] lag_weights;
  for (nn in 1:N) {
    for (media in 1 : num_media) {
      for (lag in 1 : max_lag) {
        lag_weights[max_lag-lag+1] <- pow(decay[media], (lag - 1 - peak[media]) ^ 2);
      }
     cum_effect <- Adstock(sub_col(X_media, nn, media, max_lag), lag_weights);
     X_media_adstocked[nn, media] <- log1p(cum_effect/mu_mdip[media]);
    }
  X <- append_col(X_media_adstocked, X_ctrl);
  } 
}
model {
  decay ~ beta(3,3);
  peak ~ uniform(0, ceil(max_lag/2.0));
  tau ~ normal(0, 5);
  for (i in 1 : num_media+num_ctrl) {
    beta[i] ~ normal(0, 1);
  }
  noise_var ~ inv_gamma(0.05, 0.05 * 0.01);
  y ~ normal(tau + X * beta, sqrt(noise_var));
}
'''

In [25]:
mmm_posterior = stan.build(mmm_code,data=mmm_data)

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 51, column 35: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 53, column 16: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 54, column 34: assignment operator <- is deprecated in the Stan language; use = instead.
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 56, column 4: assignment operator <- is deprecated in the Stan language; use = instead.Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 66, column 30: Argument 0.0005 suggests there may be parameters that are not unit scale; consider rescaling with a multiplier (see manual section 22.12).
Warning in '/tmp/httpstan_keqhy6qv/model_o3nw5otz.stan', line 66, column 24: Argument 0.05 suggests there may be parameters that 

In [26]:
mmm_fit = mmm_posterior.sample(num_chains=3, num_samples=20)

Sampling:   0%
Sampling:   0% (1/3600)
Sampling:   0% (2/3600)
Sampling:   0% (3/3600)